This notebook implements morphological filtering and rule-based reclassification using external layers.

### load packages

In [ ]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.coastal import get_coastlines

### input layers and set parameters

In [ ]:
output_crs='epsg:32736' # output crs: WGS84/UTM Zone 36S 
dict_map={'Tree crops':11,'Field crops':12,'Forest plantations':21,'Grassland':31,
                 'Aquatic or regularly flooded herbaceous vegetation':41,'Water body':44,
                 'Settlements':51,'Bare soils':61,'Mangrove':70,'Mecrusse':71,
                'Broadleaved (Semi-) evergreen forest':72,'Broadleaved (Semi-) deciduous forest':74,'Mopane':75} # a dictionary of pixel value for each class

# file paths
mozambique_boundary_shp='Data/Mozambique_boundary.shp'
mozambique_tiles_shp='Data/Mozambique_50km_sample_regions.geojson'
# mozambique_tiles_shp='Data/mozambique_boundaries_projected_epsg32735_tiles.shp' # mozambique tiles shapefile
river_network_shp='Data/hotosm_moz_waterways_lines_filtered.shp' # OSM river network data
road_network_shp='Data/hotosm_moz_roads_lines_filtered.shp' # OSM road network data
google_building_raster='Data/GoogleBuildingLayer_Mozambique_rasterised.tif' # google bulding layer
# classification2021_raster='Results/Land_cover_prediction_15pct_td_Mozambique_50km_sample_regions_mosaic.tif' # land cover map of 2021
classification2021_raster='Results/Land_cover_prediction_15pct_td_Mozambique_geomedian_semiannual_features_50km_sample_regions_mosaic.tif' # land cover map of 2021
hand_raster='Data/hand_Mozambique.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
wsf2019_raster='Data/WSF2019_v1_Mozambique_clipped.tif' # 2019 WSF raster

## load layers

In [ ]:
# import mozambique boundary and get bounding box
mozambique_boundary=gpd.read_file(mozambique_boundary_shp).to_crs(output_crs)

# import mozambique tiles and get bounding box
mozambique_tiles=gpd.read_file(mozambique_tiles_shp).to_crs(output_crs) # get bounding boxes of tiles covering mozambique
tile_bboxes=mozambique_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021

# load and pre-process external layers
# import OSM road network data and reproject
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) 
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
road_network_mask=xr_rasterize(gdf=road_network,da=landcover2021.squeeze(),
                               transform=landcover2021.geobox.transform,crs=output_crs) # # rasterise buffered OSM road network layer
# import OSM river network data and reproject
river_network=gpd.read_file(river_network_shp).to_crs(output_crs) 
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute
river_network_mask=xr_rasterize(gdf=river_network,da=landcover2021.squeeze(),
                                transform=landcover2021.geobox.transform,crs=output_crs) # rasterise OSM river network layer
# import hand layer
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze() 
# import google bulding layer
google_buildings=xr.open_dataset(google_building_raster,engine="rasterio").astype(np.int8).squeeze() 
wsf2019=xr.open_dataset(wsf2019_raster,engine="rasterio").astype(np.int32).squeeze() # import WSF2019 layers
# load coastline layer and buffer
shorelines_gdf = get_coastlines(mozambique_boundary.bounds.iloc[0],crs=output_crs,layer='shorelines').to_crs(output_crs)
shorelines_gdf_2021=shorelines_gdf[shorelines_gdf['year']=='2021'] # select only 2021
shorelines_gdf_2021.geometry=shorelines_gdf_2021.geometry.buffer(50000) # buffer the road network by 50km
shorelines_2021_mask=xr_rasterize(gdf=shorelines_gdf_2021,da=landcover2021.squeeze(),
                                transform=landcover2021.geobox.transform,crs=output_crs) # rasterise layer

### loop through tiles for reclassification and export as geotiffs

In [ ]:
# loop through tiles for reclassification
for i in range(0,3):
# for i in range(0,len(tile_bboxes)):
    # get tile bbox
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    
    # load s2 annual geomedian and calcualte MNDWI
    dc = datacube.Datacube(app='s2_geomedian')
    query_geomedian= {
        'time': ('2021'),
        'x': (x_min,x_max),
        'y': (y_max,y_min),
        'resolution':(-10, 10),
        'crs':output_crs,
        'output_crs': output_crs,
        'measurements':['green','swir_1']
    }
    ds_geomedian = dc.load(product="gm_s2_annual", **query_geomedian)
    ds_MNDWI = calculate_indices(ds=ds_geomedian, index='MNDWI', satellite_mission='s2',drop=True).squeeze()
    ds_geobox=ds_MNDWI.geobox # get bbox
    
    # clip land cover maps 2015 and 2021 to tile boundary
#     landcover2021_tile=landcover2021.rio.clip_box(minx=x_min,miny=y_min,maxx=x_max,maxy=y_max)
    landcover2021_tile=xr_reproject(landcover2021, ds_geobox, resampling="nearest") # clip to tile boundary
    np_landcover2021=landcover2021_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post=np_landcover2021.copy() # initialise post-processed numpy array
    
    # mode filtering for a smoother classification map
    np_landcover2021_post=modal(np_landcover2021,footprint=disk(2.5),mask=np_landcover2021!=0)
    
    # load and clip hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    river_network_mask_tile=xr_reproject(river_network_mask, ds_geobox, resampling="nearest")
    np_river_network_mask=river_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post[((np_landcover2021==dict_map['Water body'])&(np_hand<=45))|(np_river_network_mask==1)]=dict_map['Water body'] # apply rules
    
    # assign pixels overlapping google building polygons or WSF 2019 but outside difference of WSF2015 and WSF2019 as built-up
    google_buildings_mask_tile=xr_reproject(google_buildings, ds_geobox, resampling="nearest")
    np_google_buildings_mask=google_buildings_mask_tile.to_array().squeeze().to_numpy() # data array to numpy array

    wsf2019_tile=xr_reproject(wsf2019, ds_geobox, resampling="nearest") # load and clip WSF layers
    np_wsf2019=wsf2019_tile.to_array().squeeze().to_numpy()
    np_landcover2021_post[(np_google_buildings_mask==1)|(np_wsf2019==255)]=dict_map['Settlements'] # apply rules
    
    # assign pixesl overlapping buffered OSM road network as built-up class
    road_network_mask_tile=xr_reproject(road_network_mask, ds_geobox, resampling="nearest")
    np_road_network_mask=road_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post[np_road_network_mask==1]=dict_map['Settlements'] # burn in buffered OSM road network polygons
    
    # reclassify wetlands around (within 50m of) built-up areas as tree crops
    urban_buffered=binary_dilation(np_landcover2021==dict_map['Settlements'],footprint=disk(5)) # dilating built-up regions
    np_landcover2021_post[(urban_buffered==1)&(np_landcover2021==dict_map['Aquatic or regularly flooded herbaceous vegetation'])]=dict_map['Field crops'] # apply rule
    
    # reassign water using NDWI calculated from annual S2 geomedian
    np_MNDWI=ds_MNDWI['MNDWI'].to_numpy()
    np_landcover2021_post[np_MNDWI>=0]=dict_map['Water body']
    
    # reassign mangroves outside 50km of coastline as Forest Plantation
    shorelines_2021_mask_tile=xr_reproject(shorelines_2021_mask, ds_geobox, resampling="nearest") # load and clip shoreline mask layers
    np_shorelines_2021_mask=shorelines_2021_mask_tile.squeeze().to_numpy()
    np_landcover2021_post[(np_shorelines_2021_mask==0)&(np_landcover2021==dict_map['Mangrove'])]=dict_map['Forest plantations']
    
    # convert back result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    
    # export as geotiff
#     write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_postprocessed_15pct_td_Mozambique_50km_sample_region_'+str(i)+'.tif', overwrite=True)
    write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_postprocessed_15pct_td_Mozambique_geomedian_semiannual_features_50km_sample_region_'+str(i)+'.tif', overwrite=True)

### mosaic all post-processed tiles

In [6]:
! gdal_merge.py -o Results/Land_cover_prediction_postprocessed_15pct_td_Mozambique_geomedian_semiannual_features_50km_sample_regions_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover_prediction_postprocessed_15pct_td_Mozambique_geomedian_semiannual_features_50km_sample_region_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
